# Importações

In [1]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
import pandas as pd

2022-03-21 16:47:49.159356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 16:47:49.159621: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Aquisição de dados

In [3]:
from RainPredictionMachine.data import clean_data_rpm
df = clean_data_rpm()
df = df.clean_data(2)
df

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,923.1,923.1,922.1,0.0,22.0,20.5,22.2,21.8,20.5,...,92.0,172.0,2.8,0.4,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 00:00:00+00:00,nao chove
1,0.0,923.7,923.7,923.1,0.0,22.1,20.7,22.3,22.0,20.8,...,92.0,101.0,1.8,0.9,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 01:00:00+00:00,nao chove
2,0.0,923.6,923.8,923.6,0.0,23.2,20.1,23.5,22.1,20.8,...,82.0,7.0,3.3,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 02:00:00+00:00,nao chove
3,0.0,923.1,923.6,923.1,0.0,23.3,19.5,23.6,23.1,20.1,...,79.0,6.0,3.0,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 03:00:00+00:00,nao chove
4,0.0,922.6,923.1,922.6,0.0,22.9,19.0,23.6,22.8,19.5,...,79.0,345.0,3.5,0.6,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 04:00:00+00:00,nao chove
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,0.0,965.8,966.0,965.6,1636.0,26.7,22.2,27.6,26.2,23.3,...,76.0,310.0,6.3,2.4,RANCHARIA,-22.372832,-50.974710,398.75,2020-12-31 19:00:00+00:00,nao chove
8780,0.0,966.2,966.2,965.8,751.1,25.7,21.8,27.0,25.7,22.8,...,79.0,308.0,5.5,2.8,RANCHARIA,-22.372832,-50.974710,398.75,2020-12-31 20:00:00+00:00,nao chove
8781,0.0,966.6,966.6,966.0,94.1,24.6,21.8,25.7,24.6,21.8,...,84.0,312.0,5.4,1.2,RANCHARIA,-22.372832,-50.974710,398.75,2020-12-31 21:00:00+00:00,nao chove
8782,1.0,967.5,967.5,966.6,10.5,22.6,20.1,24.6,22.6,21.8,...,86.0,204.0,5.9,0.5,RANCHARIA,-22.372832,-50.974710,398.75,2020-12-31 22:00:00+00:00,fraca


# Processamento

In [4]:
from RainPredictionMachine.Trainer import pipe_creator
X_train_t, X_test_t  = pipe_creator(df)

/home/victor/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
X_train_t.shape

(12297, 20)

# construção da RNN

## normalização

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train)
normalizer(X_train)[0]

## camadas da RNN

In [ ]:
model = Sequential()
model.add(normalizer)
model.add(SimpleRNN(units=20, activation='tanh'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

## compilando

In [ ]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    return model

model = compile_model(model)

# Ajuste da RNN

Fazendo um early stopping

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

Ajustando o modelo

In [ ]:
model.fit(X_train, y_train,
          epochs=100, 
          batch_size=32, 
          validation_split=0.2,
          callbacks=[es])

In [ ]:
df


In [ ]:
df['classe_chuva'].shift(-1)